In [85]:
from bs4 import BeautifulSoup as bs
import requests
import json
from datetime import datetime
import typer
import math
import re
import sys
import csv
data_csv = {'titulo':[],'empresa':[],'descrição':[],'data de publicação':[],'salário':[],'localização':[]}
import os    
import time

API_KEY = "9fa7ce317d6e85c90d92244adb9146c6"
BASE_URL = "https://api.itjobs.pt/job/list.json"

app = typer.Typer()

def fetch_jobs(params):
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(BASE_URL, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter dados: {response.status_code} - {response.text}")
        return None
    
def top10():
    print("Procurando os 10 trabalhos mais recentes...")

    # URL base da API
    URL = 'https://api.itjobs.pt/job/list.json'
    API_KEY = '9fa7ce317d6e85c90d92244adb9146c6'  # Substitua pela sua chave de API

    # Faz a requisição para obter os trabalhos
    response = requests.get(URL, params={'api_key': API_KEY, 'limit': 10, 'page': 1}, headers={'User-Agent': 'Mozilla/5.0'})
    data = response.json()

    if 'results' in data:
        # Extraímos apenas os dados necessários: ID, título e nome da empresa
        top_10_jobs = []
        for item in data['results']:
            job_data = {
                'id': item['id'],  # ID do trabalho
                'title': item['title'],  # Nome do trabalho
                'company_name': item['company']['name']  # Nome da empresa
            }
            top_10_jobs.append(job_data)

        # Exibe o resultado em formato JSON
        print(json.dumps(top_10_jobs, indent=4))
    else:
        print("Nenhum trabalho encontrado.")


import requests
import json

def get_jobs(location, company, num_jobs):
    # API URL com a chave de API fornecida
    URL = 'https://api.itjobs.pt/job/list.json?api_key=9fa7ce317d6e85c90d92244adb9146c6'
    
    # Buscar dados da API
    try:
        request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        # Verificar se o código de status da resposta não é 200 (OK)
        if request.status_code != 200:
            print(f"Erro: Recebido código de status {request.status_code} da API")
            return
        
        # Analisar os dados JSON
        data = request.json()
        
        # Imprimir os dados brutos para inspecionar sua estrutura
        print("Dados de resposta da API:", json.dumps(data, indent=2, ensure_ascii=False))
        
        # A resposta da API tem a chave 'results' que contém os empregos
        jobs = data.get("results", [])
        
        # Verificar se 'results' contém uma lista de empregos
        if not isinstance(jobs, list):
            print("Erro: Formato inesperado dos dados da API. Esperado uma lista de empregos.")
            return
        
    except Exception as e:
        print(f"Erro ao buscar dados da API: {e}")
        return
    
    # Exibir detalhes sobre os empregos retornados
    print("\nEmpregos encontrados:")
    for job in jobs:
        company_name = job.get("company", {}).get("name", "N/A")
        job_title = job.get("title", "N/A")
        job_types = [job_type.get("name", "N/A") for job_type in job.get("types", [])]
        job_locations = [location_item.get("name", "N/A") for location_item in job.get("locations", [])]
        
        print(f"Cargo: {job_title}, Empresa: {company_name}, Tipos: {', '.join(job_types)}, Localizações: {', '.join(job_locations)}")
    
    # Filtrar os empregos com base no tipo de trabalho, localização e nome da empresa
    filtered_jobs = []
    for job in jobs:
        company_name = job.get("company", {}).get("name", "").lower()
        job_locations = [location_item.get("name", "").lower() for location_item in job.get("locations", [])]
        
        # Debug: Mostrar as informações de cada trabalho antes da filtragem
        print(f"\nVerificando trabalho: {job.get('title', 'N/A')}")
        print(f"Empresa: {company_name}, Localizações: {job_locations}")
        
        # Verificar se a empresa e a localização são correspondentes (case insensitive) e permitir correspondência parcial
        if (company.lower() in company_name) and any(location.lower() in location_item for location_item in job_locations):
            filtered_jobs.append(job)
    
    # Limitar o número de empregos ao valor especificado
    filtered_jobs = filtered_jobs[:num_jobs]
    
    # Imprimir os empregos filtrados em formato JSON
    if filtered_jobs:
        print("\nEmpregos filtrados:")
        print(json.dumps(filtered_jobs, ensure_ascii=False, indent=2))
    else:
        print("Nenhum trabalho encontrado para os critérios especificados.")
















def salary(jobID):
    URL = 'https://api.itjobs.pt/job/get.json'
    api_key = '9fa7ce317d6e85c90d92244adb9146c6'
    
    try:
        # Realiza a requisição para obter informações sobre o trabalho
        response = requests.get(URL, params={'api_key': api_key, 'id': jobID}, 
                                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        # Verifica o código de status da resposta
        if response.status_code != 200:
            print(f"Erro: Status HTTP {response.status_code}. Verifique a chave da API e o jobID.")
            return
        
        # Tenta carregar o conteúdo JSON da resposta
        data = response.json()
        
    except requests.JSONDecodeError:
        print("Erro: A resposta não está em formato JSON. Verifique a URL e os parâmetros.")
        return

    # Verifica se a resposta contém o campo 'error' indicando que o jobID não foi encontrado
    if 'error' in data:
        print(f"Erro: {data['error'].get('message', 'Erro desconhecido')}")
        return

    # Tenta extrair o salário do campo 'wage' se presente
    wage = data.get('wage')
    if wage is not None:
        print(f"Apresenta um salário inicial de {wage}€.")
        return
    
    # Caso 'wage' esteja ausente, tenta encontrar o salário em campos alternativos usando expressões regulares
    salary_found = False
    fields_to_check = ['description', 'salary_info', 'benefits']
    
    for field in fields_to_check:
        if field in data and isinstance(data[field], str):  # Verifica se o campo é uma string
            match = re.search(r'(\d+(\.\d{1,2})?)\s*(€|USD|BRL|GBP)', data[field])
            if match:
                print(f"Salário encontrado no campo {field}: {match.group(0)}")
                salary_found = True
                break
    
    # Se não encontrou o salário em nenhum dos campos alternativos
    if not salary_found:
        # Tenta fornecer informações adicionais sobre o contrato
        contract_info = []
        if 'contracts' in data and isinstance(data['contracts'], list) and data['contracts']:
            contract_info.append(f"Contrato: {data['contracts'][0].get('name', 'Desconhecido')}")
        if 'types' in data and isinstance(data['types'], list) and data['types']:
            contract_info.append(f"Tipo: {data['types'][0].get('name', 'Desconhecido')}")
        
        # Se houver informações de contrato ou tipo, exibe-as; caso contrário, informa que não há dados disponíveis
        if contract_info:
            print("Sem informação sobre o salário.\n" + ", ".join(contract_info))
        else:
            print("Sem informação sobre o trabalho selecionado.")

import requests
import re

def extract_salary_from_body(body):
    """
    Função para extrair o salário do campo 'body' usando expressões regulares.
    A função vai procurar por qualquer menção a salário.
    """
    salary_patterns = [
        r"(\d{1,3}(?:[\.,]\d{3})*(?:[\.,]\d+)?\s?(?:euros|€|bruto|neto|por mês|mensal))",  # padrão de salário em formato monetário
        r"(\d+\s?k\s?€)",  # formato de salário em k (milhares)
    ]
    
    for pattern in salary_patterns:
        match = re.search(pattern, body, re.IGNORECASE)
        if match:
            return match.group(0)
    
    return None

def get_job_salary(job_id, api_key):
    """Obtém o salário de um trabalho, verificando o campo 'wage' e, se necessário, o campo 'body'."""
    URL = 'https://api.itjobs.pt/job/get.json'
    
    # Realiza a requisição para obter o trabalho específico
    response = requests.get(URL, params={'api_key': api_key, 'id': job_id},
                            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'})
    
    # Verificar se a resposta foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao acessar a API: Status {response.status_code}")
        return
    
    try:
        data = response.json()
    except ValueError:
        print(f"Erro ao interpretar a resposta JSON.")
        return
    
    # Imprimir a resposta completa da API para ver como os dados estão sendo retornados
    print("Resposta completa da API:")
    print(data)
    
    # Verifica se o trabalho foi encontrado pela chave 'id'
    if 'id' not in data or data['id'] != int(job_id):
        print(f"Trabalho com ID {job_id} não encontrado.")
        return
    
    # Obtém o título do trabalho e a empresa
    job_title = data.get('title', 'Desconhecido')
    company_name = data['company'].get('name', 'Desconhecido')
    
    # Verifica o campo 'wage' para salário
    wage = data.get('wage')
    
    if wage:
        print(f"Salário para o trabalho '{job_title}' da empresa '{company_name}': {wage}")
    else:
        # Se o campo 'wage' estiver vazio ou ausente, procuramos no 'body'
        body = data.get('body', '')
        body_salary = extract_salary_from_body(body)
        
        if body_salary:
            print(f"Salário para o trabalho '{job_title}' da empresa '{company_name}' encontrado no campo 'body': {body_salary}")
        else:
            print(f"Não foi possível encontrar o salário para o trabalho '{job_title}' da empresa '{company_name}'.")




# Função que busca e filtra os trabalhos de acordo com as skills e o período de tempo
def skills_data(skills, data_ini, data_fim):
    not_found = 1
    URL = 'https://api.itjobs.pt/job/list.json?api_key=2fd9dd6db7e14adbf04df55811af5d22'
    
    # Fazendo a requisição inicial
    request = requests.get(URL, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
    data = request.json()
    
    alldata = []
    total_pages = math.ceil(data['total'] / 12)  # Paginação para percorrer todos os trabalhos
    
    for rep in range(total_pages):
        datasets = requests.get(URL, params={'limit': 12, 'page': rep + 1}, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}).json()
        results = datasets['results']
        
        for item in results:
            # Convertendo a data de publicação do trabalho
            data_trabalho = dt.strptime(item['publishedAt'], '%Y-%m-%d %H:%M:%S')
            
            # Verificando se a data do trabalho está dentro do período desejado
            if data_trabalho >= data_ini and data_trabalho <= data_fim:
                # Verificando se todas as habilidades exigidas estão no corpo da descrição do trabalho
                if all(re.search(skill, item['body'], re.IGNORECASE) for skill in skills):
                    # Se tudo estiver ok, guardamos os dados do trabalho
                    trabalho_info = {
                        'id': item['id'],
                        'title': item.get('title', ''),
                        'company_name': item['company'].get('name', ''),
                        'company_url': item['company'].get('url', ''),
                        'publishedAt': item.get('publishedAt', ''),
                        'wage': item.get('wage', ''),
                        'address': item['company'].get('address', '')
                    }
                    alldata.append(trabalho_info)
                    not_found = 0
    
    # Se não encontrou nenhum trabalho que satisfaça as condições
    if not_found == 1:
        return False
    
    return alldata

# Função para exibir os trabalhos conforme as skills e período
def buscar_trabalhos_por_skills(skills, data_inicial, data_final):
    # Convertendo as datas para objetos datetime
    data_ini = dt.strptime(data_inicial, '%Y-%m-%d')
    data_fim = dt.strptime(data_final, '%Y-%m-%d')

    # Chamando a função skills_data com os parâmetros recebidos
    resultados = skills_data(skills, data_ini, data_fim)
    
    # Exibindo o resultado no formato JSON
    if resultados:
        return json.dumps(resultados, indent=4)
    else:
        return "Nenhum trabalho encontrado para as habilidades e período fornecidos."















######################################################################"CSV"#################################################################################################

def top10(export_csv=False):
    print("Procurando os 10 trabalhos mais recentes...")

    # URL base da API
    URL = 'https://api.itjobs.pt/job/list.json'
    API_KEY = '9fa7ce317d6e85c90d92244adb9146c6'  # Substitua pela sua chave de API

    # Faz a requisição para obter os trabalhos
    response = requests.get(URL, params={'api_key': API_KEY, 'limit': 10, 'page': 1}, headers={'User-Agent': 'Mozilla/5.0'})
    data = response.json()

    if 'results' in data:
        # Extraímos os dados necessários
        top_10_jobs = []
        for item in data['results']:
            job_data = {
                'id': item['id'],  # ID do trabalho
                'title': item['title'],  # Nome do trabalho
                'company_name': item['company']['name'],  # Nome da empresa
                'description': item.get('description', 'N/A'),  # Descrição do trabalho
                'published_at': item.get('published_at', 'N/A'),  # Data de publicação
                'salary': item.get('salary', 'N/A'),  # Salário
                'location': ', '.join([loc['name'] for loc in item.get('locations', [])])  # Localização
            }
            top_10_jobs.append(job_data)

        # Exibe o resultado em formato JSON
        print(json.dumps(top_10_jobs, indent=4))

        # Se exportar para CSV for solicitado
        if export_csv:
            # Define o nome do arquivo CSV
            filename = f"top_10_jobs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            
            # Define os campos do CSV
            fields = ['id', 'title', 'company_name', 'description', 'published_at', 'salary', 'location']
            
            # Criação do arquivo CSV
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fields)
                
                # Escreve o cabeçalho
                writer.writeheader()
                
                # Escreve as linhas de dados
                for job in top_10_jobs:
                    writer.writerow(job)
            
            print(f"Os 10 trabalhos mais recentes foram exportados para {filename}")
    else:
        print("Nenhum trabalho encontrado.")
  







def export_to_csv(jobs, filename="jobs.csv"):
    """Função para exportar os dados dos empregos para um arquivo CSV"""
    # Cabeçalhos do CSV
    fieldnames = ['titulo', 'empresa', 'descricao', 'data_publicacao', 'salario', 'localizacao']
    
    # Criar e abrir o arquivo CSV para escrita
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Escrever o cabeçalho do CSV
        writer.writeheader()
        
        # Escrever os dados dos empregos
        for job in jobs:
            job_data = {
                'titulo': job.get('title', 'N/A'),
                'empresa': job.get('company', {}).get('name', 'N/A'),
                'descricao': job.get('body', 'N/A'),
                'data_publicacao': job.get('published', 'N/A'),
                'salario': job.get('wage', 'N/A'),
                'localizacao': ', '.join([location.get('name', 'N/A') for location in job.get('locations', [])])
            }
            writer.writerow(job_data)
    
    print(f"Dados exportados para {filename}")

def export_to_csv(jobs, filename="jobs.csv"):
    """Função para exportar os dados dos empregos para um arquivo CSV"""
    fieldnames = ['titulo', 'empresa', 'descricao', 'data_publicacao', 'salario', 'localizacao']
    
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for job in jobs:
            job_data = {
                'titulo': job.get('title', 'N/A'),
                'empresa': job.get('company', {}).get('name', 'N/A'),
                'descricao': job.get('body', 'N/A'),
                'data_publicacao': job.get('published', 'N/A'),
                'salario': job.get('wage', 'N/A'),
                'localizacao': ', '.join([location.get('name', 'N/A') for location in job.get('locations', [])])
            }
            writer.writerow(job_data)
    
    print(f"Dados exportados para {filename}")

def export_to_csv(jobs, filename="jobs.csv"):
    """Função para exportar os dados dos empregos para um arquivo CSV"""
    fieldnames = ['titulo', 'empresa', 'descricao', 'data_publicacao', 'salario', 'localizacao']
    
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for job in jobs:
            job_data = {
                'titulo': job.get('title', 'N/A'),
                'empresa': job.get('company', {}).get('name', 'N/A'),
                'descricao': job.get('body', 'N/A'),
                'data_publicacao': job.get('published', 'N/A'),
                'salario': job.get('wage', 'N/A'),
                'localizacao': ', '.join([location.get('name', 'N/A') for location in job.get('locations', [])])
            }
            writer.writerow(job_data)
    
    print(f"Dados exportados para {filename}")

def get_jobs(location, company, num_jobs, save_to_csv=False):
    # API URL com a chave de API fornecida
    URL = 'https://api.itjobs.pt/job/list.json?api_key=9fa7ce317d6e85c90d92244adb9146c6'
    
    # Buscar dados da API
    try:
        request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        # Verificar se o código de status da resposta não é 200 (OK)
        if request.status_code != 200:
            print(f"Erro: Recebido código de status {request.status_code} da API")
            return
        
        # Analisar os dados JSON
        data = request.json()
        
        # A resposta da API tem a chave 'results' que contém os empregos
        jobs = data.get("results", [])
        
        # Verificar se 'results' contém uma lista de empregos
        if not isinstance(jobs, list):
            print("Erro: Formato inesperado dos dados da API. Esperado uma lista de empregos.")
            return
        
    except Exception as e:
        print(f"Erro ao buscar dados da API: {e}")
        return
    
    # Filtrar os empregos com base no tipo de trabalho, localização e nome da empresa
    filtered_jobs = []
    for job in jobs:
        company_name = job.get("company", {}).get("name", "").lower()
        job_locations = [location_item.get("name", "").lower() for location_item in job.get("locations", [])]
        
        # Verificar se a empresa e a localização são correspondentes (case insensitive) e permitir correspondência parcial
        if (company.lower() in company_name) and any(location.lower() in location_item for location_item in job_locations):
            filtered_jobs.append(job)
    
    # Limitar o número de empregos ao valor especificado
    filtered_jobs = filtered_jobs[:num_jobs]
    
    # Exibir detalhes sobre os empregos filtrados
    if filtered_jobs:
        print("\nEmpregos filtrados:")
        for job in filtered_jobs:
            company_name = job.get("company", {}).get("name", "N/A")
            job_title = job.get("title", "N/A")
            job_description = job.get("body", "N/A")
            job_date = job.get("createdAt", "N/A")
            job_salary = job.get("wage", "N/A")
            job_locations = [location_item.get("name", "N/A") for location_item in job.get("locations", [])]
            print(f"Título: {job_title}, Empresa: {company_name}, Descrição: {job_description}, Data: {job_date}, Salário: {job_salary}, Localização: {', '.join(job_locations)}")
        
        # Se solicitado, exportar para CSV
        if save_to_csv:
            export_to_csv(filtered_jobs)
    else:
        print("Nenhum trabalho encontrado para os critérios especificados.")

def export_to_csv(jobs):
    # Nome do arquivo CSV
    filename = f"jobs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    # Cabeçalho do CSV
    header = ["Título", "Empresa", "Descrição", "Data de Publicação", "Salário", "Localização"]
    
    # Abrir o arquivo CSV para escrita
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(header)
            
            # Escrever os dados de cada emprego
            for job in jobs:
                company_name = job.get("company", {}).get("name", "N/A")
                job_title = job.get("title", "N/A")
                job_description = job.get("body", "N/A")
                job_date = job.get("createdAt", "N/A")
                job_salary = job.get("wage", "N/A")
                job_locations = [location_item.get("name", "N/A") for location_item in job.get("locations", [])]
                
                row = [job_title, company_name, job_description, job_date, job_salary, ', '.join(job_locations)]
                writer.writerow(row)
        
        print(f"\nDados exportados para o arquivo: {filename}")
    
    except Exception as e:
        print(f"Erro ao exportar para CSV: {e}")


def skills_data(skills, data_ini, data_fim, export_csv=False):
    not_found = 1
    URL = 'https://api.itjobs.pt/job/list.json?api_key=2fd9dd6db7e14adbf04df55811af5d22'
    
    # Fazendo a requisição inicial
    try:
        request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'})
        
        if request.status_code != 200:
            print(f"Erro: Recebido status code {request.status_code} da API")
            return
        
        data = request.json()
    except Exception as e:
        print(f"Erro ao buscar dados da API: {e}")
        return
    
    alldata = []
    total_pages = math.ceil(data['total'] / 12)  # Paginação para percorrer todos os trabalhos
    
    for rep in range(total_pages):
        datasets = requests.get(URL, params={'limit': 12, 'page': rep + 1}, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}).json()
        results = datasets['results']
        
        for item in results:
            # Convertendo a data de publicação do trabalho
            data_trabalho = dt.strptime(item['publishedAt'], '%Y-%m-%d %H:%M:%S')
            
            # Verificando se a data do trabalho está dentro do período desejado
            if data_trabalho >= data_ini and data_trabalho <= data_fim:
                # Verificando se todas as habilidades exigidas estão no corpo da descrição do trabalho
                if all(re.search(skill, item['body'], re.IGNORECASE) for skill in skills):
                    # Se tudo estiver ok, guardamos os dados do trabalho
                    trabalho_info = {
                        'id': item['id'],
                        'title': item.get('title', ''),
                        'company_name': item['company'].get('name', ''),
                        'company_url': item['company'].get('url', ''),
                        'publishedAt': item.get('publishedAt', ''),
                        'wage': item.get('wage', ''),
                        'address': item['company'].get('address', '')
                    }
                    alldata.append(trabalho_info)
                    not_found = 0
    
    # Se não encontrou nenhum trabalho que satisfaça as condições
    if not_found == 1:
        print("Nenhum trabalho encontrado para as habilidades e período fornecidos.")
        return None
    
    # Se exportar para CSV for solicitado
    if export_csv:
        filename = f"jobs_skills_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        # Definindo os campos para o CSV
        fields = ['id', 'title', 'company_name', 'company_url', 'publishedAt', 'wage', 'address']
        
        # Criando o arquivo CSV
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fields)
            
            # Escrevendo o cabeçalho
            writer.writeheader()
            
            # Escrevendo os dados dos trabalhos filtrados
            for job in alldata:
                writer.writerow(job)
        
        print(f"Os trabalhos filtrados foram exportados para {filename}")
    
    return json.dumps(alldata, indent=4)

# Função para exibir os trabalhos conforme as skills e período
def buscar_trabalhos_por_skills(skills, data_inicial, data_final, export_csv=False):
    # Convertendo as datas para objetos datetime
    data_ini = dt.strptime(data_inicial, '%Y-%m-%d')
    data_fim = dt.strptime(data_final, '%Y-%m-%d')

    # Chamando a função skills_data com os parâmetros recebidos
    resultados = skills_data(skills, data_ini, data_fim, export_csv)
    
    # Exibindo o resultado no formato JSON
    if resultados:
        return resultados
    else:
        return "Nenhum trabalho encontrado para as habilidades e período fornecidos."